In [16]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd

In [17]:
%run ./ml_model_init.ipynb

## Baseline Model


In [18]:
#
rc_data= data[(data['minutes_5'] >= 300) & (data['position'] != 'GK')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'was_home', 'ict_index_3',
                                                                         'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5', 'threat_3', 'threat_5',
                                                                         'red_cards', 'red_cards_3', 'red_cards_5', 'whh', 'whd', 'wha']]

rc_data_tar = data_tar[(data_tar['minutes_5'] >= 300) & (data_tar['position'] != 'GK')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'was_home', 'ict_index_3',
                                                                         'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5', 'threat_3', 'threat_5',
                                                                         'red_cards', 'red_cards_3', 'red_cards_5', 'whh', 'whd', 'wha']]

# players = data_225 #.drop('position', axis=1) #  & (data['position'] == 'FWD')
# # players


In [19]:

rc = rc_data[['red_cards']]
feats = rc_data.drop('red_cards', axis=1)
feats_train, feats_test, rc_train, rc_test = train_test_split(feats, rc , test_size=0.2, random_state=42)

In [20]:
# Split the gk data into train and test sets
# for_splits = split_data(fwd_player_data)

#### Linear Model


In [21]:

lin_reg = Linear_regression(feats_train, feats_test, rc_train, rc_test)

# Store the model evaluation details in a DataFrame
evaluation_stats = pd.DataFrame({"lin_reg": [lin_reg['train_MAE'], lin_reg['test_MAE'], lin_reg['train_RMSE'], lin_reg['test_RMSE'], lin_reg['cv_rmse'], lin_reg['R2_train'], lin_reg['R2_test']]},
                                                        index=(['train_MAE','test_MAE', 'train_RMSE', 'test_RMSE', 'cv_rmse', 'R2_train', 'R2_test']))

evaluation_stats

Training set RMSE: 0.06853227889370518
Test set RMSE: 0.07526525632004363
Training set R2: 0.0024042790310668805
Test set R2: -0.003971196260609489


,lin_reg
train_MAE,0.009752
test_MAE,0.010828
train_RMSE,0.068532
test_RMSE,0.075265
cv_rmse,0.063443
R2_train,0.002404
R2_test,-0.003971


#### DecisionTree Model


In [22]:
dt_reg = DecisionTreeRegression(feats_train, feats_test, rc_train, rc_test)

# Store the model evaluation details in a DataFrame
evaluation_stats = evaluation_stats.assign(dt_reg = [dt_reg['train_MAE'], dt_reg['test_MAE'], dt_reg['train_RMSE'], dt_reg['test_RMSE'], dt_reg['cv_rmse'], dt_reg['R2_train'], dt_reg['R2_test']])

evaluation_stats



,lin_reg,dt_reg
train_MAE,0.009752,2.690148e-17
test_MAE,0.010828,1.197982e-02
train_RMSE,0.068532,3.254895e-17
test_RMSE,0.075265,1.094524e-01
cv_rmse,0.063443,9.677457e-02
R2_train,0.002404,1.000000e+00
R2_test,-0.003971,-1.123159e+00


#### RandomForest Model


In [23]:
hyperparameters = {"criterion": 'friedman_mse', "max_depth": 8, "max_features": 'sqrt', "n_estimators": 20}
rf_reg = RandomForestRegression(feats_train, feats_test, rc_train, rc_test, hyperparameters)

# Store the model evaluation details in a DataFrame
evaluation_stats = evaluation_stats.assign(rf_reg = [rf_reg['train_MAE'], rf_reg['test_MAE'], rf_reg['train_RMSE'], rf_reg['test_RMSE'], rf_reg['cv_rmse'], rf_reg['R2_train'], rf_reg['R2_test']])

evaluation_stats

,lin_reg,dt_reg,rf_reg
train_MAE,0.009752,2.690148e-17,0.006597
test_MAE,0.010828,1.197982e-02,0.011323
train_RMSE,0.068532,3.254895e-17,0.047521
test_RMSE,0.075265,1.094524e-01,0.076860
cv_rmse,0.063443,9.677457e-02,0.067512
R2_train,0.002404,1.000000e+00,0.520329
R2_test,-0.003971,-1.123159e+00,-0.046980


#### XgBoost Model


In [24]:
hyperparameters = {'learning_rate': 0.02, 'max_depth': 4, 'n_estimators': 150}
xgb_reg = XGBoostRegression(feats_train, feats_test, rc_train, rc_test, hyperparameters)

# Store the model evaluation details in a DataFrame
evaluation_stats = evaluation_stats.assign(xgb_reg = [xgb_reg['train_MAE'], xgb_reg['test_MAE'], xgb_reg['train_RMSE'], xgb_reg['test_RMSE'], xgb_reg['cv_rmse'], xgb_reg['R2_train'], xgb_reg['R2_test']])
evaluation_stats

,lin_reg,dt_reg,rf_reg,xgb_reg
train_MAE,0.009752,2.690148e-17,0.006597,0.008567
test_MAE,0.010828,1.197982e-02,0.011323,0.010025
train_RMSE,0.068532,3.254895e-17,0.047521,0.064179
test_RMSE,0.075265,1.094524e-01,0.076860,0.075344
cv_rmse,0.063443,9.677457e-02,0.067512,0.065109
R2_train,0.002404,1.000000e+00,0.520329,0.125108
R2_test,-0.003971,-1.123159e+00,-0.046980,-0.006070


# Feature engineering


### VarianceThreshold


In [25]:
from sklearn.compose import TransformedTargetRegressor, ColumnTransformer
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import pandas as pd

# Feature preprocessing pipeline
# Split into numerical and categorical columns
bool_cols = feats_train.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_cols = feats_train.select_dtypes(include=['object', 'category']).columns.tolist()

# Preprocessing for numerical features
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Preprocessing for categorical features
categorical_transformer = Pipeline(steps=[
    ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessors in a column transformer
col_trans = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, bool_cols),
        ('cat', categorical_transformer, categorical_cols),
    ]
)

# Scale and remove low-variance features
preprocessor = Pipeline(steps=[
    ('col_trans', col_trans),
    ('var_threshold', VarianceThreshold(threshold=0.1))
])




#### Model(VT)


In [ ]:
def scores(model, feats_train, feats_test, rc_train, rc_test):
    # fit the transofrmer on the train data
    model.fit(feats_train, rc_train)

    support = model.regressor_.named_steps['preprocessor'].named_steps['var_threshold'].get_support()
    # print(f"Selected features: {feats_train.columns[support]}")
    print(len(support), len(feats_train.columns))

    # With the model fitted, we can predict the total_points given the feature_train and feature_test set
    pred_train = model.predict(feats_train)
    pred_test = model.predict(feats_test)

    # Evaluate the performance of the model on both sets using the mean absolute error
    train_MAE = mean_absolute_error(rc_train, pred_train)
    test_MAE = mean_absolute_error(rc_test, pred_test)

    # Evaluate the performance of the model on both sets using the mean square error
    train_MSE = mean_squared_error(rc_train, pred_train)
    test_MSE = mean_squared_error(rc_test, pred_test)

    # Evaluate the performance of the model on both sets using the root mean square error
    train_RMSE = mean_squared_error(rc_train, pred_train, squared=False)
    test_RMSE = mean_squared_error(rc_test, pred_test, squared=False)

    # Get the score of the model or the coeeficient of determination i.e how much of the target value can be explained by the model.
    # In this case, 0.6 implies that 60% of the variations in the target value can be explained by the model and 40% is not explainable
    R2_train = model.score(feats_train, rc_train)
    R2_test = model.score(feats_test, rc_test)

    # If the test error significantly differs from the train error, then there is either overfitting or underfitting
    # RMSE, just like the squared loss function that it derives from, effectively penalizes larger errors more severely.
    print('Training set RMSE: {}'.format(train_RMSE))
    print('Test set RMSE: {}'.format(test_RMSE))

    print('Training set R2: {}'.format(R2_train))
    print('Test set R2: {}'.format(R2_test))

    # Carry out cross validation of the model.
    # The evaluation method is the root mean square error
    # The method expects a utility function (greater is better) and so the scoring function is the opposite of the the RMSE. Hence the -ve
    tree_rmses = -cross_val_score(model, feats_train, rc_train,
                                    scoring="neg_root_mean_squared_error", cv=10)

    return {'train_MAE': train_MAE, 'test_MAE': test_MAE, 'train_MSE': train_MSE, 'test_MSE': test_MSE, 'train_RMSE': train_RMSE, 'test_RMSE': test_RMSE, 'cv_rmse': tree_rmses.mean(), 'R2_train': R2_train, 'R2_test': R2_test}


##### Linear Model


In [ ]:

# Full pipeline including the regression model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

# TransformedTargetRegressor for target scaling
model = TransformedTargetRegressor(
    regressor=pipeline,
    transformer=StandardScaler()
)

# Model predictions
VarT_lin_reg = scores(model, feats_train, feats_test, rc_train, rc_test)
# Store the model evaluation details in a DataFrame
evaluation_stats = evaluation_stats.assign(VarT_lin_reg =  [VarT_lin_reg['train_MAE'], VarT_lin_reg['test_MAE'],  VarT_lin_reg['train_RMSE'], VarT_lin_reg['test_RMSE'], VarT_lin_reg['cv_rmse'], VarT_lin_reg['R2_train'], VarT_lin_reg['R2_test']])

evaluation_stats

20 19
Training set RMSE: 0.06853227889370518
Test set RMSE: 0.07526525632004363
Training set R2: 0.0024042790310668805
Test set R2: -0.003971196260609489


,lin_reg,dt_reg,rf_reg,xgb_reg,VarT_lin_reg
train_MAE,0.009752,2.690148e-17,0.006597,0.008567,0.009752
test_MAE,0.010828,1.197982e-02,0.011323,0.010025,0.010828
train_RMSE,0.068532,3.254895e-17,0.047521,0.064179,0.068532
test_RMSE,0.075265,1.094524e-01,0.076860,0.075344,0.075265
cv_rmse,0.063443,9.677457e-02,0.067512,0.065109,0.063443
R2_train,0.002404,1.000000e+00,0.520329,0.125108,0.002404
R2_test,-0.003971,-1.123159e+00,-0.046980,-0.006070,-0.003971


##### DecisionTree Model


In [ ]:
# Full pipeline including the regression model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', DecisionTreeRegressor())
])

# TransformedTargetRegressor for target scaling
dt_model = TransformedTargetRegressor(
    regressor=pipeline,
    transformer=StandardScaler()
)




VarT_dt_reg = scores(dt_model, feats_train, feats_test, rc_train, rc_test)

# Store the model evaluation details in a DataFrame
evaluation_stats = evaluation_stats.assign(VarT_dt_reg = [VarT_dt_reg['train_MAE'], VarT_dt_reg['test_MAE'], VarT_dt_reg['train_RMSE'], VarT_dt_reg['test_RMSE'], VarT_dt_reg['cv_rmse'], VarT_dt_reg['R2_train'], VarT_dt_reg['R2_test']])

evaluation_stats



20 19
Training set RMSE: 3.0347173072919816e-17
Test set RMSE: 0.11229577231219766
Training set R2: 1.0
Test set R2: -1.2349045304023107


,lin_reg,dt_reg,rf_reg,xgb_reg,VarT_lin_reg,VarT_dt_reg
train_MAE,0.009752,2.690148e-17,0.006597,0.008567,0.009752,2.237539e-17
test_MAE,0.010828,1.197982e-02,0.011323,0.010025,0.010828,1.261034e-02
train_RMSE,0.068532,3.254895e-17,0.047521,0.064179,0.068532,3.034717e-17
test_RMSE,0.075265,1.094524e-01,0.076860,0.075344,0.075265,1.122958e-01
cv_rmse,0.063443,9.677457e-02,0.067512,0.065109,0.063443,1.032512e-01
R2_train,0.002404,1.000000e+00,0.520329,0.125108,0.002404,1.000000e+00
R2_test,-0.003971,-1.123159e+00,-0.046980,-0.006070,-0.003971,-1.234905e+00


##### RandomForest Model


In [ ]:
hyperparameters = {"criterion": 'friedman_mse', "max_depth": 8, "max_features": 'sqrt', "n_estimators": 20}

# Full pipeline including the regression model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(
                        n_estimators=hyperparameters['n_estimators'],
                        max_depth=hyperparameters['max_depth'],
                        criterion=hyperparameters['criterion'], random_state=18
                        ))
])

# TransformedTargetRegressor for target scaling
rf_model = TransformedTargetRegressor(
    regressor=pipeline,
    transformer=StandardScaler()
)
# Store the model evaluation details in a DataFrame
VarT_rf_reg = scores(rf_model, feats_train, feats_test, rc_train, rc_test)

# Store the model evaluation details in a DataFrame
evaluation_stats = evaluation_stats.assign(VarT_rf_reg = [VarT_rf_reg['train_MAE'], VarT_rf_reg['test_MAE'], VarT_rf_reg['train_RMSE'], VarT_rf_reg['test_RMSE'], VarT_rf_reg['cv_rmse'], VarT_rf_reg['R2_train'], VarT_rf_reg['R2_test']])
evaluation_stats

20 19
Training set RMSE: 0.04748005851763884
Test set RMSE: 0.0765691502234695
Training set R2: 0.5211640801479553
Test set R2: -0.03905806516712973


,lin_reg,dt_reg,rf_reg,xgb_reg,VarT_lin_reg,VarT_dt_reg,VarT_rf_reg
train_MAE,0.009752,2.690148e-17,0.006597,0.008567,0.009752,2.237539e-17,0.006611
test_MAE,0.010828,1.197982e-02,0.011323,0.010025,0.010828,1.261034e-02,0.011181
train_RMSE,0.068532,3.254895e-17,0.047521,0.064179,0.068532,3.034717e-17,0.047480
test_RMSE,0.075265,1.094524e-01,0.076860,0.075344,0.075265,1.122958e-01,0.076569
cv_rmse,0.063443,9.677457e-02,0.067512,0.065109,0.063443,1.032512e-01,0.068008
R2_train,0.002404,1.000000e+00,0.520329,0.125108,0.002404,1.000000e+00,0.521164
R2_test,-0.003971,-1.123159e+00,-0.046980,-0.006070,-0.003971,-1.234905e+00,-0.039058


##### XgBoost Model


In [ ]:
hyperparameters = {'learning_rate': 0.02, 'max_depth': 4, 'n_estimators': 150}

# Full pipeline including the regression model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', xgb(learning_rate=hyperparameters["learning_rate"],
                    n_estimators=hyperparameters["n_estimators"],
                    max_depth=hyperparameters["max_depth"],
                    eval_metric='rmsle'))
])

# TransformedTargetRegressor for target scaling
rf_model = TransformedTargetRegressor(
    regressor=pipeline,
    transformer=StandardScaler()
)

VarT_xgb_reg = scores(rf_model, feats_train, feats_test, rc_train, rc_test)



# Store the model evaluation details in a DataFrame
evaluation_stats = evaluation_stats.assign(VarT_xgb_reg = [VarT_xgb_reg['train_MAE'], VarT_xgb_reg['test_MAE'], VarT_xgb_reg['train_RMSE'], VarT_xgb_reg['test_RMSE'], VarT_xgb_reg['cv_rmse'], VarT_xgb_reg['R2_train'], VarT_xgb_reg['R2_test']])
evaluation_stats

20 19
Training set RMSE: 0.06417933659488004
Test set RMSE: 0.07534389584464521
Training set R2: 0.12510753206522918
Test set R2: -0.0060702541029697965


,lin_reg,dt_reg,rf_reg,xgb_reg,VarT_lin_reg,VarT_dt_reg,VarT_rf_reg,VarT_xgb_reg
train_MAE,0.009752,2.690148e-17,0.006597,0.008567,0.009752,2.237539e-17,0.006611,0.008567
test_MAE,0.010828,1.197982e-02,0.011323,0.010025,0.010828,1.261034e-02,0.011181,0.010025
train_RMSE,0.068532,3.254895e-17,0.047521,0.064179,0.068532,3.034717e-17,0.047480,0.064179
test_RMSE,0.075265,1.094524e-01,0.076860,0.075344,0.075265,1.122958e-01,0.076569,0.075344
cv_rmse,0.063443,9.677457e-02,0.067512,0.065109,0.063443,1.032512e-01,0.068008,0.065109
R2_train,0.002404,1.000000e+00,0.520329,0.125108,0.002404,1.000000e+00,0.521164,0.125108
R2_test,-0.003971,-1.123159e+00,-0.046980,-0.006070,-0.003971,-1.234905e+00,-0.039058,-0.006070
